In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import date

In [2]:
path = Path.home() / 'OneDrive - Seagroup/kaggle_dataset/OnlineRetail.csv'
df = pd.read_csv(path, encoding='ISO-8859-1')

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['date_id'] = pd.to_datetime(df['InvoiceDate']).dt.strftime('%Y-%m-%d')
df.eval('amount = UnitPrice * Quantity', inplace=True)

print(df.shape)
df.head()

(541909, 10)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date_id,amount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34


In [3]:
tx_6m = df[(df['date_id'] < '2011-09-01') & (df['date_id'] >= '2011-03-01')].reset_index(drop=True)
tx_next = df[(df['date_id'] >= '2011-09-01') & (df['date_id'] < '2011-12-01')].reset_index(drop=True)

In [4]:
tx_next_first_purchase = tx_next.groupby('CustomerID').InvoiceDate.min().rename('MinPurchaseDate').reset_index()
tx_last_purchase = tx_6m.groupby('CustomerID').InvoiceDate.max().rename('MaxPurchaseDate').reset_index()

tx_purchase_dates = pd.merge(tx_last_purchase, tx_next_first_purchase, on='CustomerID', how='left')
tx_purchase_dates['NextPurchaseDay'] = (tx_purchase_dates['MinPurchaseDate'] - tx_purchase_dates['MaxPurchaseDate']).dt.days
tx_purchase_dates.head()

,CustomerID,MaxPurchaseDate,MinPurchaseDate,NextPurchaseDay
0,12347.0,2011-08-02 08:48:00,2011-10-31 12:25:00,90.0
1,12348.0,2011-04-05 10:47:00,2011-09-25 13:13:00,173.0
2,12352.0,2011-03-22 16:08:00,2011-09-20 14:34:00,181.0
3,12353.0,2011-05-19 17:47:00,NaT,NaN
4,12354.0,2011-04-21 13:11:00,NaT,NaN
